In [14]:
import time
from pytket.circuit import Circuit, CircBox, OpType, QControlBox
from pytket.extensions.qiskit import AerBackend
from pytket.passes import DecomposeBoxes, SynthesiseTket

In [15]:
backend = AerBackend()

from pytket.circuit import Circuit, CircBox
import numpy as np
from pytket.circuit import ConjugationBox

def build_qft_circuit(n_qubits: int, do_swaps: True) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    if do_swaps:
        for k in range(0, n_qubits // 2):
            circ.SWAP(k, n_qubits - k - 1)
    return circ

class DraperQFTAdderConstantPytket:
    def __init__(self, num_state_qubits: int, constant: int) -> Circuit:

        qft_circuit = build_qft_circuit(num_state_qubits, do_swaps=False)

        compute = CircBox(qft_circuit)

        # Apply phase rotations for the constant
        circuit = Circuit(num_state_qubits)
        for qubit in range(num_state_qubits):
            angle = (constant % (2 ** (qubit + 1))) * np.pi / (2 ** qubit)
            circuit.Rz(angle, qubit)
        action = CircBox(circuit)

        self.circuit_box = ConjugationBox(compute, action)

    def get_circuit_box(self) -> Circuit:
        return self.circuit_box

In [16]:
def get_reflect_around_zero(size):
    compute = CircBox(Circuit(size).X(0).H(0))
    # qc.X(0)
    # qc.H(0)
    qc = Circuit(size, name="reflection")
    qc.add_gate(QControlBox(CircBox(Circuit(1).X(0)), n_controls = size-1, control_state=[0]*(size-1)), [k for k in range(size)])
    action = CircBox(qc)

    return ConjugationBox(compute, action)


def get_cir_be(qc, data, block):
    qc.H(block[0])
    qc.H(block[2])
    adder_2_cir = DraperQFTAdderConstantPytket(len(data)+1, 2).get_circuit_box()
    adder_1_cir = DraperQFTAdderConstantPytket(len(data)+1, -1+2**(len(data)+1)).get_circuit_box()
    qc.add_gate(QControlBox(adder_2_cir, n_controls = 1, control_state=0), [block[0]]+[data[k] for k in range(len(data))]+[block[1]])
    qc.add_gate(adder_1_cir, [data[k] for k in range(len(data))]+[block[1]])
    qc.add_gate(QControlBox(get_reflect_around_zero(len(data)),n_controls=1, control_state=0),[block[2]]+[data[k] for k in range(len(data))])
    qc.H(block[0])
    qc.H(block[2])

    return qc

def apply_projector_controlled_phase(qc, phase, block_reg, aux_reg):
    qc.add_gate(QControlBox(CircBox(Circuit(1).X(0)), n_controls = len(block_reg), control_state=[0]*len(block_reg)),
                                         [block_reg[k] for k in range(len(block_reg))] + [aux_reg[0]]
                                         )
    qc.Rz(phase, aux_reg[0])

    qc.add_gate(QControlBox(CircBox(Circuit(1).X(0)), n_controls = len(block_reg), control_state=[0]*len(block_reg)),
                                         [block_reg[k] for k in range(len(block_reg))] + [aux_reg[0]]
                                         )


def apply_qsvt_step(qc, phase1, phase2, u, data, block, qsvt_aux):

    qc.add_gate(CircBox(u), [data[l] for l in range(len(data))]+ [block[l] for l in range(len(block))])
    apply_projector_controlled_phase(qc, phase1, block, qsvt_aux)
    qc.add_gate(CircBox(u).dagger, [data[l] for l in range(len(data))]+ [block[l] for l in range(len(block))])
    apply_projector_controlled_phase(qc, phase2, block, qsvt_aux)


def get_qsvt_circuit(qsvt_phases,
            size):

    qsvt_cir = Circuit()
    data = qsvt_cir.add_q_register("data", size)
    block = qsvt_cir.add_q_register("block", 3)
    cir_be = Circuit()
    data = cir_be.add_q_register("data", size)
    block = cir_be.add_q_register("block", 3)

    qsvt_aux = qsvt_cir.add_q_register("qsvt_aux", 1)
    cir_be = get_cir_be(cir_be,data, block)

    qsvt_cir.H(qsvt_aux[0])
    apply_projector_controlled_phase(qsvt_cir, qsvt_phases[0], block, qsvt_aux)
    for i in range(int(np.floor((len(qsvt_phases) - 1) / 2))):
        apply_qsvt_step(qsvt_cir,
                        qsvt_phases[(2 * i) + 1], qsvt_phases[(2 * i) + 2],
                        cir_be,
                        data,
                        block,
                        qsvt_aux
                       )
    qsvt_cir.add_gate(CircBox(cir_be), [data[l] for l in range(len(data))]+ [block[l] for l in range(len(block))])
    apply_projector_controlled_phase(qsvt_cir, qsvt_phases[len(qsvt_phases) - 1], block, qsvt_aux)
    qsvt_cir.H(qsvt_aux[0])

    return qsvt_cir

In [19]:
SIZES = list(range(5, 21))
DEGREE = 3
QSVT_PHASES = [1.280311896404252, 8.127145628464149, 1.8439603212845617, -5.002873410775335]
save_dict = {}
import pandas as pd


for SIZE in SIZES:

    start_time = time.time()
    qc_qsvt = get_qsvt_circuit(QSVT_PHASES, SIZE)
    DecomposeBoxes().apply(qc_qsvt)
    SynthesiseTket().apply(qc_qsvt)
    compiled_circ = backend.get_compiled_circuit(qc_qsvt)
    transpilation_time = time.time()-start_time
    depth = compiled_circ.depth()
    cx_counts = compiled_circ.n_gates_of_type(OpType.CX)
    width = compiled_circ.n_qubits

    print(f"==== tket for {SIZE}==== time: {transpilation_time}")
    print(f"depth: {depth}")
    print(f"cx_counts: {cx_counts}")
    print(f"width: {width}")
    print(compiled_circ)

    save_dict[SIZE] = {"time": transpilation_time, "depth": depth, "cx_counts": cx_counts, "width": width}
    pd.DataFrame(save_dict).to_csv("tket_nathan_qsvt.csv")



==== tket for 5==== time: 11.95797085762024
depth: 918
cx_counts: 716
width: 9
<tket::Circuit, qubits=9, gates=1492>
==== tket for 6==== time: 13.533084869384766
depth: 1138
cx_counts: 982
width: 10
<tket::Circuit, qubits=10, gates=2035>
==== tket for 7==== time: 17.21721386909485
depth: 1388
cx_counts: 1304
width: 11
<tket::Circuit, qubits=11, gates=2693>
==== tket for 8==== time: 21.147334814071655
depth: 1656
cx_counts: 1674
width: 12
<tket::Circuit, qubits=12, gates=3445>
==== tket for 9==== time: 26.68986678123474
depth: 1954
cx_counts: 2092
width: 13
<tket::Circuit, qubits=13, gates=4294>
==== tket for 10==== time: 35.265530824661255
depth: 2270
cx_counts: 2558
width: 14
<tket::Circuit, qubits=14, gates=5238>
==== tket for 11==== time: 40.35980010032654
depth: 2616
cx_counts: 3072
width: 15
<tket::Circuit, qubits=15, gates=6277>
==== tket for 12==== time: 47.99701976776123
depth: 2980
cx_counts: 3634
width: 16
<tket::Circuit, qubits=16, gates=7414>
==== tket for 13==== time: 55.4